In [1]:
# Cell 1: Import all required libraries
import sys
import time
import os
from pathlib import Path
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from scipy import optimize
from scipy.interpolate import griddata

# Pymoo imports for NSGA-II
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.termination import get_termination
from pymoo.optimize import minimize
from pymoo.core.callback import Callback
from pymoo.util.display.column import Column
from pymoo.util.misc import termination_from_tuple

# Standard libraries for file handling
import json
import csv
import warnings
warnings.filterwarnings('ignore')  # Suppress warnings

# Additional math and stats
import math
from statistics import mean, median

# 1) Locate project root (one level up from this notebooks directory)
try:
    project_root = Path(__file__).resolve().parent.parent
except NameError:
    project_root = Path.cwd().parent
    print(f"Project root identified as: {project_root}")

# 2) Point at your scripts folder - UPDATED PATH
scripts_dir = project_root / "5_nsga" / "5_nsga_scripts_master"
if not scripts_dir.exists():
    # Try alternatives if the first path doesn't exist
    alternatives = [
        project_root / "5_nsga_scripts_master",
        Path.cwd() / "5_nsga_scripts_master",
        Path.cwd().parent / "5_nsga" / "5_nsga_scripts_master"
    ]
    
    for alt in alternatives:
        if alt.exists():
            scripts_dir = alt
            break
    else:
        raise FileNotFoundError(f"Cannot find scripts directory. Tried: {scripts_dir} and alternatives: {alternatives}")

print(f"Using scripts directory: {scripts_dir}")
sys.path.insert(0, str(scripts_dir))

# Import custom modules
try:
    from battery import simulate_battery_dispatch
    from pv import simulate_multi_year_pv
    from fin import compute_financials
    from obj import evaluate_solution
    print("All modules imported successfully")
except ImportError as e:
    print(f"Error importing modules: {e}")
    print(f"Files in scripts directory: {[f.name for f in scripts_dir.iterdir() if f.is_file()]}")
    raise

print("All imports completed successfully")

Project root identified as: /Users/petertunali/Documents/GitHub/Battery_Optimisation
Using scripts directory: /Users/petertunali/Documents/GitHub/Battery_Optimisation/5_nsga/5_nsga_scripts_master
All modules imported successfully
All imports completed successfully


In [2]:
# Cell 2: Set up output directories
# Set up results folder in the same directory as the master notebook
results_dir = scripts_dir / "5_nsga_results"
results_dir.mkdir(exist_ok=True)

# Create a new numbered subfolder for this run
existing_runs = [d for d in results_dir.iterdir() if d.is_dir() and d.name.isdigit()]
next_run_num = 1 if not existing_runs else max([int(d.name) for d in existing_runs]) + 1
run_dir = results_dir / f"{next_run_num:03d}"
run_dir.mkdir(exist_ok=True)

print(f"Results will be saved to: {run_dir}")

Results will be saved to: /Users/petertunali/Documents/GitHub/Battery_Optimisation/5_nsga/5_nsga_scripts_master/5_nsga_results/002


In [3]:
# cell 3
# Data directory path - SIMPLIFIED APPROACH
# Looking at your repository structure
data_dir = Path("/Users/petertunali/Documents/GitHub/Battery_Optimisation/data")
if not data_dir.exists():
    alternatives = [
        project_root / "data",
        project_root.parent / "Battery_Optimisation" / "data",
        Path.cwd() / "data"
    ]
    
    for alt in alternatives:
        if alt.exists():
            data_dir = alt
            break
    else:
        print("\nWARNING: Data directory not found at expected locations.")
        print("Please enter the absolute path to your data directory:")
        user_path = input().strip()
        data_dir = Path(user_path)
        if not data_dir.exists():
            raise FileNotFoundError(f"Data directory not found: {data_dir}\nPlease check your path and try again.")

print(f"Using data directory: {data_dir}")

# List files in the data directory to verify
print("\nFiles in data directory:")
try:
    for file in sorted(data_dir.iterdir()):
        if file.is_file():
            print(f"  - {file.name}")
except Exception as e:
    print(f"Error listing files: {e}")

# Use specific weather files in the correct order
print("\nLooking for weather files...")
desired_weather_files = [
    "Bonfire_2025.epw",
    "Bonfire_2040_4_5.epw",
    "Bonfire_2050_4_5.epw"
]

weather_files = []
for filename in desired_weather_files:
    file_path = data_dir / filename
    if file_path.exists():
        weather_files.append(str(file_path))
        print(f"  ✔ Found: {filename}")
    else:
        print(f"  ❌ Missing: {filename}")

if len(weather_files) < 3:
    # If we didn't find all the specific files, look for any .epw files
    print("\nSearching for alternative .epw files...")
    available_epw = sorted(list(data_dir.glob("*.epw")))
    
    if available_epw:
        for epw in available_epw:
            if str(epw) not in weather_files:
                print(f"  Found alternative: {epw.name}")
        
        # If we need more files to reach 3, use alternatives
        while len(weather_files) < 3 and available_epw:
            for epw in available_epw:
                if str(epw) not in weather_files:
                    weather_files.append(str(epw))
                    print(f"  Using: {epw.name}")
                    break
    
    # If we still don't have 3 weather files, we need to check the path
    if len(weather_files) < 3:
        raise FileNotFoundError(
            f"Could not find the required weather files: {', '.join(desired_weather_files)}\n"
            f"Please ensure these files exist in: {data_dir}"
        )

print("\nUsing these weather files:")
for i, wf in enumerate(weather_files):
    print(f"  {i+1}. {Path(wf).name}")

# Check for demand files
demand_paths = [
    data_dir / "PV_Generation_excel.csv",
    data_dir / "Energy_Demand_and_Supply_2024.csv"
]

for p in demand_paths:
    if p.exists():
        demand_file = p
        print(f"\n✔ Found demand file: {p}")
        break
else:
    # List available CSV files so user can see what's available
    csv_files = sorted(list(data_dir.glob("*.csv")))
    if csv_files:
        print("\nNo demand file found with expected name. Available CSV files:")
        for csv in csv_files:
            print(f"  - {csv.name}")
        raise FileNotFoundError(
            "Expected demand files not found. Please check file names."
        )
    else:
        raise FileNotFoundError(
            f"No CSV files found in {data_dir}. Please check your data directory."
        )

# Rest of the code remains the same
# Define existing PV system
existing_pv = {
    'name': 'existing_system',
    'system_capacity_kw': 10.0,
    'tilt': 10.0,
    'azimuth': 18.0,
    'shading': 43.0,
    'array_type': 1  # Roof-mounted
}

# Define new PV system options based on priority
pv_options = [
    {
        'name': 'accommodation_block',
        'max_capacity_kw': 33.0,
        'tilt': 20.0,
        'azimuth': 40.0,
        'shading': 0.0,
        'array_type': 1,  # Roof-mounted
        'cost_multiplier': 1.0
    },
    {
        'name': 'small_shed',
        'max_capacity_kw': 10.0,
        'tilt': 20.0,
        'azimuth': 20.0,
        'shading': 20.0,
        'array_type': 1,  # Roof-mounted
        'cost_multiplier': 1.0
    },
    {
        'name': 'ground_mounted',
        'max_capacity_kw': float('inf'),  # Unlimited
        'tilt': 30.0,
        'azimuth': 5.0,
        'shading': 0.0,
        'array_type': 0,  # Ground-mounted
        'cost_multiplier': 1.2  # 20% cost increase
    }
]

print("\nExisting PV system:")
print(f"  - {existing_pv['name']}: {existing_pv['system_capacity_kw']} kW, Tilt: {existing_pv['tilt']}°, Azimuth: {existing_pv['azimuth']}°, Shading: {existing_pv['shading']}%")

print("\nNew PV options:")
for option in pv_options:
    print(f"  - {option['name']}: Max {option['max_capacity_kw']} kW, Tilt: {option['tilt']}°, Azimuth: {option['azimuth']}°, Shading: {option['shading']}%, Cost multiplier: {option['cost_multiplier']}")

Using data directory: /Users/petertunali/Documents/GitHub/Battery_Optimisation/data

Files in data directory:
  - .DS_Store
  - Analysis_import Cost.xlsx
  - Bonfire_2025.epw
  - Bonfire_2040_4_5.epw
  - Bonfire_2040_RCP8_5.epw
  - Bonfire_2050_4_5.epw
  - Bonfire_2050_RCP8_5.epw
  - Energy_Data_2024.xlsx
  - Energy_Demand_and_Supply_2024.csv
  - PV_Generation_excel.csv
  - PV_Generation_excel.xlsx
  - PV_Generation_excel_2024.csv
  - demand_without.csv
  - ~$Energy_Data_2024.xlsx

Looking for weather files...
  ✔ Found: Bonfire_2025.epw
  ✔ Found: Bonfire_2040_4_5.epw
  ✔ Found: Bonfire_2050_4_5.epw

Using these weather files:
  1. Bonfire_2025.epw
  2. Bonfire_2040_4_5.epw
  3. Bonfire_2050_4_5.epw

✔ Found demand file: /Users/petertunali/Documents/GitHub/Battery_Optimisation/data/PV_Generation_excel.csv

Existing PV system:
  - existing_system: 10.0 kW, Tilt: 10.0°, Azimuth: 18.0°, Shading: 43.0%

New PV options:
  - accommodation_block: Max 33.0 kW, Tilt: 20.0°, Azimuth: 40.0°, Sha

In [4]:
# Cell 4: Define utility functions for the optimization
import pandas as pd
from pathlib import Path
import numpy as np

def load_demand_profile(csv_path: Path) -> pd.Series:
    """Load demand profile from CSV file."""
    # Load the CSV and skip NaN timestamp rows immediately
    raw = pd.read_csv(csv_path, parse_dates=['Date and Time'], dayfirst=True)
    
    # Drop rows with NaN timestamps right away
    raw = raw.dropna(subset=['Date and Time'])
    print(f"CSV loaded with {len(raw)} valid rows")
    
    # Get the consumption column
    consumption_col = 'Consumtpion (kWh)'  # Based on your data sample
    if consumption_col not in raw.columns:
        # Try to find the consumption column
        for col in raw.columns:
            if 'consum' in col.lower() or 'demand' in col.lower():
                consumption_col = col
                break
        else:
            # If no consumption column found, use the second column
            consumption_col = raw.columns[1]
    
    print(f"Using consumption column: '{consumption_col}'")
    
    # Create a Series with timestamp index and consumption values
    s = pd.Series(raw[consumption_col].values, index=raw['Date and Time'])
    
    # Check for duplicate timestamps
    dup_count = s.index.duplicated().sum()
    if dup_count:
        print(f"⚠️ Dropping {dup_count} duplicate timestamps")
        s = s[~s.index.duplicated(keep='first')]
    
    # Build the expected half-hour index for entire year (no Feb 29)
    year = s.index.min().year
    start = pd.Timestamp(year, 1, 1, 0, 0)
    end = pd.Timestamp(year, 12, 31, 23, 30)
    expected = pd.date_range(start, end, freq="30min")
    expected = expected[~((expected.month==2) & (expected.day==29))]
    
    # Reindex to ensure complete coverage
    s = s.reindex(expected)
    missing = s.isna().sum()
    if missing:
        print(f"⚠️ Filling {missing} missing points with 0")
        s = s.fillna(0.0)
    
    # Final sanity check
    assert len(s) == 17520, f"Got {len(s)} points, expected 17520"
    return s

def create_30_year_profile(one_year_series: pd.Series) -> pd.Series:
    """Create a 30-year profile from a 1-year series."""
    start_year = one_year_series.index[0].year
    years = 30
    all_data = []
    
    for year_offset in range(years):
        # Copy the data for this year
        year_data = one_year_series.copy()
        
        # Create index for this specific year
        year_start = pd.Timestamp(start_year + year_offset, 1, 1, 0, 0)
        year_end = pd.Timestamp(start_year + year_offset, 12, 31, 23, 30)
        year_range = pd.date_range(start=year_start, end=year_end, freq="30min")
        
        # Remove Feb 29 if it's a leap year
        year_range = year_range[~((year_range.month == 2) & (year_range.day == 29))]
        
        # Make sure it has the right number of points
        assert len(year_range) == len(one_year_series), f"Year {start_year + year_offset} has {len(year_range)} points, expected {len(one_year_series)}"
        
        # Assign the new index and add to our list
        year_data.index = year_range
        all_data.append(year_data)
    
    # Concatenate all years
    return pd.concat(all_data)

def calculate_pv_cost(capacity_kw: float, cost_multiplier: float = 1.0) -> float:
    """
    Calculate PV cost using the formula: y = 1047.3 * e^(-0.002*x) with minimum $750
    
    Args:
        capacity_kw: PV capacity in kW
        cost_multiplier: Multiplier for special installations (e.g., 1.2 for ground-mounted)
        
    Returns:
        cost_per_kw: Cost per kW in dollars
    """
    if capacity_kw <= 0:
        return 0.0
    
    # Apply economies of scale formula
    cost_per_kw = 1047.3 * np.exp(-0.002 * capacity_kw)
    
    # Apply minimum cost of $750/kW
    cost_per_kw = max(750.0, cost_per_kw)
    
    # Apply cost multiplier
    return cost_per_kw * cost_multiplier

def allocate_pv_capacity(total_capacity_kw: float, options: list) -> list:
    """
    Allocate PV capacity across available options based on priority.
    
    Args:
        total_capacity_kw: Total additional PV capacity to allocate
        options: List of PV options with max_capacity_kw and other parameters
        
    Returns:
        allocated_pv: List of PV configurations with allocated capacity
    """
    remaining_capacity = total_capacity_kw
    allocated_pv = []
    
    for option in options:
        option_copy = option.copy()
        # Allocate capacity to this option (limited by max capacity)
        allocation = min(remaining_capacity, option['max_capacity_kw'])
        
        if allocation > 0:
            option_copy['system_capacity_kw'] = allocation
            allocated_pv.append(option_copy)
            remaining_capacity -= allocation
        
        if remaining_capacity <= 0:
            break
    
    return allocated_pv

# Test PV cost function
print("\nTesting PV cost function:")
test_capacities = [5, 10, 20, 30, 40, 50, 100]
print("Capacity (kW) | Cost per kW ($)")
print("--------------------------")
for cap in test_capacities:
    cost = calculate_pv_cost(cap)
    print(f"{cap:12.1f} | ${cost:10.2f}")

# Test ground-mounted with 20% premium
print("\nGround-mounted PV with 20% premium:")
print("Capacity (kW) | Cost per kW ($)")
print("--------------------------")
for cap in [10, 30, 50]:
    cost = calculate_pv_cost(cap, cost_multiplier=1.2)
    print(f"{cap:12.1f} | ${cost:10.2f}")

# Test PV allocation
print("\nTesting PV allocation:")
print("Testing with 50 kW total capacity...")
test_options = [option.copy() for option in pv_options]
allocated = allocate_pv_capacity(50, test_options)
print("Allocation results:")
for pv in allocated:
    print(f"  - {pv['name']}: {pv['system_capacity_kw']:.2f} kW")

print("\nUtility functions defined and tested successfully!")


Testing PV cost function:
Capacity (kW) | Cost per kW ($)
--------------------------
         5.0 | $   1036.88
        10.0 | $   1026.56
        20.0 | $   1006.23
        30.0 | $    986.31
        40.0 | $    966.78
        50.0 | $    947.64
       100.0 | $    857.46

Ground-mounted PV with 20% premium:
Capacity (kW) | Cost per kW ($)
--------------------------
        10.0 | $   1231.87
        30.0 | $   1183.57
        50.0 | $   1137.16

Testing PV allocation:
Testing with 50 kW total capacity...
Allocation results:
  - accommodation_block: 33.00 kW
  - small_shed: 10.00 kW
  - ground_mounted: 7.00 kW

Utility functions defined and tested successfully!


In [5]:
# Cell 5: Load one-year demand and build 30-year profile
print(f"Loading demand data from: {demand_file}")
one_year_demand = load_demand_profile(demand_file)
print(f"One year data points: {len(one_year_demand)}")

# Create 30-year demand profile
demand_profile = create_30_year_profile(one_year_demand)
print("\n30-year demand profile built:")
print(f"  • Time steps : {len(demand_profile)}")
print(f"  • Date range : {demand_profile.index[0]} → {demand_profile.index[-1]}")
print(f"  • Total demand: {demand_profile.sum():.2f} kWh")

Loading demand data from: /Users/petertunali/Documents/GitHub/Battery_Optimisation/data/PV_Generation_excel.csv
CSV loaded with 17520 valid rows
Using consumption column: 'Consumtpion (kWh)'
One year data points: 17520

30-year demand profile built:
  • Time steps : 525600
  • Date range : 2025-01-01 00:00:00 → 2054-12-31 23:30:00
  • Total demand: 1318609.49 kWh


In [6]:
# Cell 6: Simulate existing PV system
import time  # Add this import
from datetime import timedelta  # Also add this if you're using timedelta

print("\nSimulating 30-year PV generation for existing system...")
start_time = time.time()
start_years = [2025, 2040, 2050]

# Simulate existing PV system
pv_profile = simulate_multi_year_pv(
    weather_files=weather_files,
    roof_params=[existing_pv],
    repeats_per_file=10,
    start_years=start_years
)

elapsed = time.time() - start_time
print(f"PV simulation completed in {elapsed:.1f} seconds ({timedelta(seconds=int(elapsed))})")
print(f"   • PV steps: {len(pv_profile)}")
print(f"   • Date range: {pv_profile.index[0]} → {pv_profile.index[-1]}")
print(f"   • Total generation: {pv_profile['simulated_kwh'].sum():.2f} kWh")


Simulating 30-year PV generation for existing system...
PV simulation completed in 0.2 seconds (0:00:00)
   • PV steps: 525600
   • Date range: 2025-01-01 00:00:00 → 2054-12-24 23:30:00
   • Total generation: 204028.14 kWh


In [7]:
print("\nComputing baseline costs...")
start_time = time.time()

# Define actual electricity costs from your electricity bill data
ANNUAL_NO_PV_COST = 9424.48      # From your bill - no PV, no battery
ANNUAL_PV_ONLY_COST = 8246.44    # From your bill - with PV, no battery

# Calculate 30-year baseline costs
NO_PV_BASELINE_COST = ANNUAL_NO_PV_COST * 30
PV_ONLY_BASELINE_COST = ANNUAL_PV_ONLY_COST * 30
print(f"Annual no-PV cost: ${ANNUAL_NO_PV_COST:,.2f}")
print(f"Annual PV-only cost: ${ANNUAL_PV_ONLY_COST:,.2f}")
print(f"Annual savings from PV alone: ${ANNUAL_NO_PV_COST - ANNUAL_PV_ONLY_COST:,.2f}")
print(f"30-year no-PV baseline: ${NO_PV_BASELINE_COST:,.2f}")
print(f"30-year PV-only baseline: ${PV_ONLY_BASELINE_COST:,.2f}")

# Define electricity rates and escalation rate
base_peak_rate = 0.39710        # Peak rate
base_offpeak_rate = 0.13530     # Off-peak rate  
base_export_rate = 0.033        # Feed-in tariff
escalation_rate = 0.03          # 3% annual price escalation

# Simulate with existing PV, no battery to verify
disp0, totals0 = simulate_battery_dispatch(
    pv_gen=pv_profile['simulated_kwh'],
    demand=demand_profile,
    battery_kwh=0.0,
    roundtrip_eff=0.9,
    min_soc_pct=0.05,
    annual_deg_rate=0.01,  # Fallback rate if NREL model fails
    grid_emission_rate=0.81
)

# Calculate PV-only annual cost from simulation with 3% annual escalation
# First year cost (without escalation)
first_year_import_cost = (
    totals0['total_grid_import_peak'] * base_peak_rate / 30 + 
    totals0['total_grid_import_offpeak'] * base_offpeak_rate / 30
)
first_year_export_revenue = totals0['total_pv_export'] * base_export_rate / 30
first_year_net_cost = first_year_import_cost - first_year_export_revenue

# Calculate 30-year cost with escalation
total_cost_with_escalation = 0
for year in range(30):
    year_escalation = (1 + escalation_rate)**year
    year_cost = first_year_net_cost * year_escalation
    total_cost_with_escalation += year_cost

simulated_pv_only_annual_cost = first_year_net_cost
simulated_pv_only_total_cost = total_cost_with_escalation

print(f"\nSimulated first-year PV-only cost: ${simulated_pv_only_annual_cost:,.2f}")
print(f"Actual annual PV-only cost from bill: ${ANNUAL_PV_ONLY_COST:,.2f}")
print(f"Difference: ${simulated_pv_only_annual_cost - ANNUAL_PV_ONLY_COST:,.2f}")

# Use actual values from bill for optimization
BASE_IMPORT_COST = NO_PV_BASELINE_COST
PV_ONLY_COST = PV_ONLY_BASELINE_COST

print(f"\n   → Baseline no-PV cost over 30 yr = ${BASE_IMPORT_COST:,.2f}")
print(f"   → Baseline PV-only cost over 30 yr = ${PV_ONLY_COST:,.2f}")
elapsed = time.time() - start_time
print(f"Baseline calculation completed in {elapsed:.1f} seconds ({timedelta(seconds=int(elapsed))})")

# Store these values for the objective function
import obj
obj.BASE_IMPORT_COST = BASE_IMPORT_COST
obj.PV_ONLY_COST = PV_ONLY_COST
obj.WEATHER_FILES = weather_files
obj.START_YEARS = start_years
obj.EXISTING_PV_CAPACITY = existing_pv['system_capacity_kw']
obj.PV_OPTIONS = pv_options
obj.BASE_PEAK_RATE = base_peak_rate
obj.BASE_OFFPEAK_RATE = base_offpeak_rate
obj.BASE_EXPORT_RATE = base_export_rate
obj.ESCALATION_RATE = escalation_rate


Computing baseline costs...
Annual no-PV cost: $9,424.48
Annual PV-only cost: $8,246.44
Annual savings from PV alone: $1,178.04
30-year no-PV baseline: $282,734.40
30-year PV-only baseline: $247,393.20

Simulated first-year PV-only cost: $8,249.58
Actual annual PV-only cost from bill: $8,246.44
Difference: $3.14

   → Baseline no-PV cost over 30 yr = $282,734.40
   → Baseline PV-only cost over 30 yr = $247,393.20
Baseline calculation completed in 1.0 seconds (0:00:01)


In [8]:
# Cell 7.5: Ultra-simplified Battery Economics Analysis (No Simulation)
print("\nPerforming ultra-simplified battery economics analysis (NO SIMULATION)...")

# Calculate battery costs for different sizes
print("\nBattery cost examples:")
print("Size (kWh) | Cost per kWh ($) | Total Cost ($) | Installation ($) | Total ($)")
print("-----------|-----------------|----------------|-------------------|----------")
for size in [0, 5, 10, 20, 50, 100]:
    if size <= 0:
        cost_per_kwh = 0
        total_cost = 0
        install_cost = 0
        all_in_cost = 0
    else:
        cost_per_kwh = 977.54 * np.exp(-0.004 * size)
        cost_per_kwh = max(600.0, cost_per_kwh)
        total_cost = size * cost_per_kwh
        install_cost = size * 174.0
        all_in_cost = total_cost + install_cost
    
    print(f"{size:10d} | ${cost_per_kwh:15.2f} | ${total_cost:14,.2f} | ${install_cost:17,.2f} | ${all_in_cost:8,.2f}")

# Skip the simulation and use estimated values for a 10 kWh battery
print("\nEstimated values for a 10 kWh battery (NO ACTUAL SIMULATION):")
batt_size = 10.0

# Estimated values based on typical performance
estimated_cycles = 3000
estimated_degradation = 25
estimated_self_consumption = 85
estimated_renewable_fraction = 15

# Use estimated electricity costs (from observed patterns)
estimated_annual_savings = 250  # $250/year savings vs PV-only
estimated_annual_electricity_cost = ANNUAL_PV_ONLY_COST - estimated_annual_savings
battery_capex = batt_size * (max(600.0, 977.54 * np.exp(-0.004 * batt_size)) + 174.0)
simple_payback = battery_capex / estimated_annual_savings if estimated_annual_savings > 0 else float('inf')

print("\nEstimated Economic Analysis Results:")
print(f"Battery size: {batt_size} kWh")
print(f"Battery capital cost: ${battery_capex:,.2f}")
print(f"Estimated annual electricity cost: ${estimated_annual_electricity_cost:,.2f}/year")
print(f"Estimated annual savings vs. PV-only: ${estimated_annual_savings:,.2f}/year")
if simple_payback < float('inf'):
    print(f"Estimated simple payback period: {simple_payback:.1f} years")
else:
    print("Simple payback: N/A (no positive savings)")

# Show estimated technical performance
print("\nEstimated Technical Performance Metrics:")
print(f"Estimated battery cycles over 30 years: {estimated_cycles:.1f}")
print(f"Estimated final battery degradation: {estimated_degradation:.1f}%")
print(f"Estimated self-consumption rate: {estimated_self_consumption:.1f}%")
print(f"Estimated renewable fraction: {estimated_renewable_fraction:.1f}%")

# Sample calculation for first-year cash flow with 3% escalation
batt_cost_per_kwh = max(600.0, 977.54 * np.exp(-0.004 * batt_size))
print("\nSample Economics With Electricity Price Escalation (3%/year):")
print("Year | Electricity Cost | PV-only Cost | Savings | Cumulative Savings")
print("-----|-----------------|--------------|---------|-------------------")
cumulative = -battery_capex  # Start with initial investment
for year in range(5):  # Just show first 5 years
    escalation = (1 + escalation_rate)**year
    year_cost = estimated_annual_electricity_cost * escalation
    pv_only_cost = ANNUAL_PV_ONLY_COST * escalation
    savings = pv_only_cost - year_cost
    cumulative += savings
    print(f"{year+1:4d} | ${year_cost:15,.2f} | ${pv_only_cost:12,.2f} | ${savings:7,.2f} | ${cumulative:17,.2f}")

# Calculate simplified IRR and NPV
cash_flows = [-battery_capex]
for year in range(30):
    escalation = (1 + escalation_rate)**year
    year_cost = estimated_annual_electricity_cost * escalation
    pv_only_cost = ANNUAL_PV_ONLY_COST * escalation
    savings = pv_only_cost - year_cost
    cash_flows.append(savings)

# Calculate NPV with 7% discount rate
npv = sum(cf / (1 + 0.07)**(i) for i, cf in enumerate(cash_flows))

# Calculate IRR
try:
    from scipy import optimize
    def npv_func(rate):
        return sum(cf / (1 + rate)**(i) for i, cf in enumerate(cash_flows))
    
    try:
        irr = optimize.newton(npv_func, 0.05)
        print(f"\nEstimated Internal Rate of Return (IRR): {irr*100:.2f}%")
    except:
        print("\nEstimated IRR: Could not calculate (likely negative or undefined)")
except:
    print("\nIRR calculation failed - scipy.optimize not available")

print(f"Estimated Net Present Value (NPV): ${npv:,.2f}")

# PV Cost Model example
print("\nPV Cost Model Example (with economies of scale):")
print("Capacity (kW) | Base Cost ($/kW) | Ground-mounted ($/kW)")
print("--------------|------------------|---------------------")
for capacity in [5, 10, 20, 50, 100]:
    base_cost = calculate_pv_cost(capacity)
    ground_cost = calculate_pv_cost(capacity, cost_multiplier=1.2)
    print(f"{capacity:12d} | ${base_cost:16.2f} | ${ground_cost:19.2f}")

# Explain the PV allocation strategy
print("\nPV Allocation Strategy:")
print("When additional PV capacity is added, it's allocated in this priority order:")
for i, option in enumerate(pv_options):
    print(f"{i+1}. {option['name']}: up to {option['max_capacity_kw'] if option['max_capacity_kw'] != float('inf') else 'unlimited'} kW max")
    print(f"   - Tilt: {option['tilt']}°, Azimuth: {option['azimuth']}°, Shading: {option['shading']}%")
    if option['cost_multiplier'] != 1.0:
        print(f"   - Price multiplier: {option['cost_multiplier']:.1f}x")

print("\nSimplified analysis complete. Ready to proceed with full NSGA-II optimization.")


Performing ultra-simplified battery economics analysis (NO SIMULATION)...

Battery cost examples:
Size (kWh) | Cost per kWh ($) | Total Cost ($) | Installation ($) | Total ($)
-----------|-----------------|----------------|-------------------|----------
         0 | $           0.00 | $          0.00 | $             0.00 | $    0.00
         5 | $         958.18 | $      4,790.92 | $           870.00 | $5,660.92
        10 | $         939.21 | $      9,392.10 | $         1,740.00 | $11,132.10
        20 | $         902.38 | $     18,047.66 | $         3,480.00 | $21,527.66
        50 | $         800.34 | $     40,017.10 | $         8,700.00 | $48,717.10
       100 | $         655.26 | $     65,526.47 | $        17,400.00 | $82,926.47

Estimated values for a 10 kWh battery (NO ACTUAL SIMULATION):

Estimated Economic Analysis Results:
Battery size: 10.0 kWh
Battery capital cost: $11,132.10
Estimated annual electricity cost: $7,996.44/year
Estimated annual savings vs. PV-only: $250.00/ye

In [9]:
# Cell 8: Define the NSGA-II optimization problem
print("Defining optimization problem with battery + additional PV (IRR and NPV only)...")
try:
    # Define the NSGA-II optimization problem
    class BatteryPVOptimizationProblem(Problem):
        def __init__(self):
            # Define two variables:
            # 1. Battery size (kWh) - 0 to 100 kWh
            # 2. Additional PV capacity (kW) - 0 to 100 kW
            super().__init__(
                n_var=2,               # Number of decision variables
                n_obj=2,               # Number of objectives (IRR, NPV)
                n_ieq_constr=0,        # Number of inequality constraints
                xl=np.array([0.0, 0.0]),  # Lower bounds
                xu=np.array([100.0, 100.0]) # Upper bounds
            )
            # Store the profiles for use in evaluation
            self.pv_profile = pv_profile
            self.demand_profile = demand_profile
        
        def _evaluate(self, x, out, *args, **kwargs):
            # Evaluate each solution in the population
            n_solutions = x.shape[0]
            F = np.zeros((n_solutions, 2))  # For storing [-IRR, -NPV]
            
            # Loop through each solution
            for i in range(n_solutions):
                battery_kwh = x[i, 0]
                additional_pv_kw = x[i, 1]
                
                # Define parameters for evaluation
                params = {
                    'battery_kwh': battery_kwh,
                    'additional_pv_kw': additional_pv_kw
                }
                
                # Get the evaluation results (IRR and NPV)
                result = evaluate_solution(params, self.pv_profile, self.demand_profile)
                F[i, :] = result
            
            # Set the output objectives array
            out["F"] = F
    
    # Create an instance of the problem
    problem = BatteryPVOptimizationProblem()
    print("Optimization problem defined successfully")
    
except Exception as e:
    print(f"Error defining optimization problem: {e}")
    raise

Defining optimization problem with battery + additional PV (IRR and NPV only)...
Optimization problem defined successfully


In [10]:
# Cell 9: Define callback for tracking optimization progress
class BestSolutionCallback(Callback):
    def __init__(self):
        super().__init__()
        self.data = {
            "gen": [],
            "best_irr": [],
            "best_npv": [],
            "batt_irr": [],
            "batt_npv": [],
            "pv_irr": [],
            "pv_npv": [],
            "time_elapsed": []
        }
        self.start_time = time.time()
        self.last_print = self.start_time
    
    def notify(self, algorithm):
        gen = algorithm.n_gen
        self.data["gen"].append(gen)
        
        # Get current best objectives
        F = algorithm.pop.get("F")
        best_irr = -np.min(F[:, 0])  # Convert back from -IRR
        best_npv = -np.min(F[:, 1])  # Convert back from -NPV
        
        self.data["best_irr"].append(best_irr)
        self.data["best_npv"].append(best_npv)
        
        # Calculate time statistics
        elapsed = time.time() - self.start_time
        self.data["time_elapsed"].append(elapsed)
        
        # Calculate average time per generation
        avg_time_per_gen = elapsed / gen if gen > 0 else 0
        
        # Estimate time remaining
        n_generations = algorithm.termination.n_max_gen
        remaining_gens = n_generations - gen
        est_remaining_time = avg_time_per_gen * remaining_gens
        
        # Find battery and PV sizes for best IRR and NPV
        X = algorithm.pop.get("X")
        best_irr_idx = np.argmin(F[:, 0])
        best_npv_idx = np.argmin(F[:, 1])
        
        best_irr_batt = X[best_irr_idx, 0]
        best_irr_pv = X[best_irr_idx, 1]
        best_npv_batt = X[best_npv_idx, 0]
        best_npv_pv = X[best_npv_idx, 1]
        
        # Store values for best IRR and NPV
        self.data["batt_irr"].append(best_irr_batt)
        self.data["batt_npv"].append(best_npv_batt)
        self.data["pv_irr"].append(best_irr_pv)
        self.data["pv_npv"].append(best_npv_pv)
        
        # Log progress every 5 generations or if more than 30 seconds passed since last print
        current_time = time.time()
        time_since_last_print = current_time - self.last_print
        
        if gen % 5 == 0 or gen == 1 or time_since_last_print > 30:
            print(f"Generation {gen:3d}/{n_generations}: " 
                  f"Best IRR = {best_irr*100:7.2f}% (Batt: {best_irr_batt:.1f} kWh, PV: {best_irr_pv:.1f} kW), " 
                  f"Best NPV = ${best_npv:10,.2f} (Batt: {best_npv_batt:.1f} kWh, PV: {best_npv_pv:.1f} kW), "
                  f"Time: {timedelta(seconds=int(elapsed))}, "
                  f"Est. remaining: {timedelta(seconds=int(est_remaining_time))}")
            self.last_print = current_time


In [11]:
# Cell 10: Run NSGA-II optimization with simplified settings
print("Running NSGA-II optimization with simplified battery model...")
total_start_time = time.time()
callback = BestSolutionCallback()

# Reduced population size and number of generations to avoid memory issues
pop_size = 30  # Reduced from 50
n_generations = 30  # Reduced from 60
print(f"Population size: {pop_size}, Generations: {n_generations}")
print(f"Total evaluations: {pop_size * n_generations}")

try:
    res = minimize(
        problem,
        NSGA2(pop_size=pop_size),
        get_termination("n_gen", n_generations),
        seed=42,
        verbose=False,
        callback=callback
    )
    
    # Extract & save Pareto front
    solutions = res.X
    battery_sizes = solutions[:, 0]
    additional_pv = solutions[:, 1]
    pareto_F = res.F
    irr_vals = -pareto_F[:, 0]  # Convert from -IRR to IRR (higher is better)
    npv_vals = -pareto_F[:, 1]  # Convert from -NPV to NPV (higher is better)
    
    # Create Pareto front DataFrame
    df = pd.DataFrame({
        'battery_kwh': battery_sizes,
        'additional_pv_kw': additional_pv,
        'IRR': irr_vals,
        'NPV': npv_vals
    })
    
    # Add PV allocation details
    allocation_details = []
    for pv_kw in additional_pv:
        allocated_pv = allocate_pv_capacity(pv_kw, pv_options)
        details = {
            'total_additional_pv': pv_kw,
            'total_system_pv': existing_pv['system_capacity_kw'] + pv_kw
        }
        
        # Add allocation for each option
        for option in pv_options:
            option_name = option['name']
            allocated = next((p['system_capacity_kw'] for p in allocated_pv if p['name'] == option_name), 0.0)
            details[f'{option_name}_kw'] = allocated
        
        allocation_details.append(details)
    
    # Convert allocation details to DataFrame and join with main results
    allocation_df = pd.DataFrame(allocation_details)
    df = pd.concat([df, allocation_df], axis=1)
    
    # Total runtime
    total_elapsed = time.time() - total_start_time
    print(f"\nOptimization complete! Total runtime: {timedelta(seconds=int(total_elapsed))}")
    print(f"Average time per evaluation: {total_elapsed/(pop_size * n_generations):.3f} seconds")
    
    # Save results
    df.to_csv(run_dir / 'pareto_solutions.csv', index=False)
    print(f"✅ Pareto front saved to {run_dir/'pareto_solutions.csv'}")
    
    # Display best solutions for each objective
    print("\nBest solutions found:")
    print(f"Best IRR: {df['IRR'].max()*100:.2f}% with {df.loc[df['IRR'].idxmax(), 'battery_kwh']:.1f} kWh battery and {df.loc[df['IRR'].idxmax(), 'total_additional_pv']:.1f} kW additional PV")
    print(f"Best NPV: ${df['NPV'].max():,.2f} with {df.loc[df['NPV'].idxmax(), 'battery_kwh']:.1f} kWh battery and {df.loc[df['NPV'].idxmax(), 'total_additional_pv']:.1f} kW additional PV")

    # Optional: Calculate emissions for just the best solutions to avoid crashes
    print("\nCalculating emissions for best solutions only...")
    
    # Just calculate emissions for the best IRR and NPV solutions
    best_solutions = [
        df.loc[df['IRR'].idxmax()],
        df.loc[df['NPV'].idxmax()]
    ]
    
    for i, solution in enumerate(best_solutions):
        print(f"Calculating emissions for {'Best IRR' if i==0 else 'Best NPV'} solution...")
        
        params = {
            'battery_kwh': solution['battery_kwh'],
            'additional_pv_kw': solution['additional_pv_kw']
        }
        
        # Simulate with these parameters to get emissions
        allocated_pv = allocate_pv_capacity(params['additional_pv_kw'], pv_options)
        
        # Add existing PV
        all_pv = [existing_pv.copy()] + allocated_pv
        
        try:
            # Simulate combined PV
            combined_pv = simulate_multi_year_pv(
                weather_files=weather_files,
                roof_params=all_pv,
                repeats_per_file=10,
                start_years=start_years
            )
            
            # Simulate battery dispatch
            _, totals = simulate_battery_dispatch(
                pv_gen=combined_pv['simulated_kwh'],
                demand=demand_profile,
                battery_kwh=params['battery_kwh'],
                battery_kw=params['battery_kwh'] * 0.5,
                roundtrip_eff=0.9,
                min_soc_pct=0.05,
                annual_deg_rate=0.01,
                grid_emission_rate=0.81
            )
            
            print(f"  Emissions: {totals['total_grid_emissions']:,.2f} kg CO2e")
            print(f"  Renewable Fraction: {totals['renewable_fraction']*100:.2f}%")
            print(f"  Self-Consumption Rate: {totals['self_consumption_rate']*100:.2f}%")
            
            # For Best IRR solution, save detailed outputs
            if i == 0:
                solution_details = {
                    'renewable_fraction': totals['renewable_fraction'],
                    'self_consumption_rate': totals['self_consumption_rate'],
                    'total_grid_emissions': totals['total_grid_emissions'],
                    'total_grid_import': totals['total_grid_import_peak'] + totals['total_grid_import_offpeak'],
                    'total_pv_export': totals['total_pv_export'],
                    'battery_cycles': totals['battery_cycles'],
                    'final_degradation_pct': totals['final_degradation_pct']
                }
                
                # Save details to CSV
                pd.DataFrame([solution_details]).to_csv(run_dir / 'best_irr_solution_details.csv', index=False)
                print(f"✅ Best IRR solution details saved to {run_dir/'best_irr_solution_details.csv'}")
        except Exception as e:
            print(f"Error calculating emissions: {e}")

except Exception as e:
    print(f"Error during optimization: {e}")
    import traceback
    traceback.print_exc()

Running NSGA-II optimization with simplified battery model...
Population size: 30, Generations: 30
Total evaluations: 900
Generation   1/30: Best IRR =   13.70% (Batt: 4.5 kWh, PV: 32.5 kW), Best NPV = $ 40,704.78 (Batt: 20.0 kWh, PV: 51.4 kW), Time: 0:00:47, Est. remaining: 0:22:53
Generation   2/30: Best IRR =   13.83% (Batt: 3.4 kWh, PV: 32.0 kW), Best NPV = $ 40,704.78 (Batt: 20.0 kWh, PV: 51.4 kW), Time: 0:01:36, Est. remaining: 0:22:32
Generation   3/30: Best IRR =   13.87% (Batt: 5.6 kWh, PV: 20.6 kW), Best NPV = $ 40,704.78 (Batt: 20.0 kWh, PV: 51.4 kW), Time: 0:02:23, Est. remaining: 0:21:35
Generation   4/30: Best IRR =   14.09% (Batt: 4.2 kWh, PV: 17.6 kW), Best NPV = $ 40,707.86 (Batt: 19.9 kWh, PV: 51.4 kW), Time: 0:03:08, Est. remaining: 0:20:28
Generation   5/30: Best IRR =   14.28% (Batt: 3.4 kWh, PV: 19.6 kW), Best NPV = $ 40,715.45 (Batt: 19.9 kWh, PV: 51.1 kW), Time: 0:03:54, Est. remaining: 0:19:32
Generation   6/30: Best IRR =   14.29% (Batt: 3.3 kWh, PV: 19.6 kW),

In [13]:
# Cell 11: Create visualization of Pareto front
# Find best IRR and NPV indices
best_irr_idx = df['IRR'].idxmax()
best_npv_idx = df['NPV'].idxmax()

# Create a scatter plot of the Pareto front
plt.figure(figsize=(10, 6))
scatter = plt.scatter(df['IRR']*100, df['NPV'], 
                     c=df['battery_kwh'], s=df['additional_pv_kw']*5, 
                     cmap='viridis', alpha=0.7)

plt.colorbar(scatter, label='Battery Size (kWh)')
plt.xlabel('IRR (%)')
plt.ylabel('NPV ($)')
plt.title('Pareto Front: IRR vs NPV')
plt.grid(True, linestyle='--', alpha=0.7)

# Annotate the best points
plt.annotate(f"Best IRR: {df['IRR'].max()*100:.1f}%\nBatt: {df.loc[best_irr_idx, 'battery_kwh']:.1f}kWh\nPV: {df.loc[best_irr_idx, 'additional_pv_kw']:.1f}kW",
             xy=(df.loc[best_irr_idx, 'IRR']*100, df.loc[best_irr_idx, 'NPV']),
             xytext=(10, -30), textcoords='offset points',
             arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=.2'))

plt.annotate(f"Best NPV: ${df['NPV'].max():,.0f}\nBatt: {df.loc[best_npv_idx, 'battery_kwh']:.1f}kWh\nPV: {df.loc[best_npv_idx, 'additional_pv_kw']:.1f}kW",
             xy=(df.loc[best_npv_idx, 'IRR']*100, df.loc[best_npv_idx, 'NPV']),
             xytext=(-70, 30), textcoords='offset points',
             arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=.2'))

plt.tight_layout()
plt.savefig(run_dir / 'pareto_front_irr_npv.png', dpi=300)
plt.close()
print(f"✅ Pareto front visualization saved to {run_dir/'pareto_front_irr_npv.png'}")

# Save convergence data
convergence_df = pd.DataFrame({
    'Generation': callback.data['gen'],
    'Best_IRR': [irr * 100 for irr in callback.data['best_irr']],
    'Best_NPV': callback.data['best_npv'],
    'Battery_IRR': callback.data['batt_irr'],
    'Battery_NPV': callback.data['batt_npv'],
    'PV_IRR': callback.data['pv_irr'],
    'PV_NPV': callback.data['pv_npv'],
    'Time_Elapsed': callback.data['time_elapsed']
})
convergence_df.to_csv(run_dir / 'convergence_data.csv', index=False)
print(f"✅ Convergence data saved to {run_dir/'convergence_data.csv'}")

# Create convergence plot
plt.figure(figsize=(12, 8))

# Create two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

# Plot IRR convergence
ax1.plot(callback.data['gen'], [irr * 100 for irr in callback.data['best_irr']], 'b-', linewidth=2)
ax1.set_ylabel('Best IRR (%)')
ax1.set_title('NSGA-II Convergence: IRR and NPV')
ax1.grid(True, linestyle='--', alpha=0.7)

# Plot NPV convergence
ax2.plot(callback.data['gen'], callback.data['best_npv'], 'g-', linewidth=2)
ax2.set_xlabel('Generation')
ax2.set_ylabel('Best NPV ($)')
ax2.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig(run_dir / 'convergence_plot.png', dpi=300)
plt.close()
print(f"✅ Convergence plot saved to {run_dir/'convergence_plot.png'}")

✅ Pareto front visualization saved to /Users/petertunali/Documents/GitHub/Battery_Optimisation/5_nsga/5_nsga_scripts_master/5_nsga_results/002/pareto_front_irr_npv.png
✅ Convergence data saved to /Users/petertunali/Documents/GitHub/Battery_Optimisation/5_nsga/5_nsga_scripts_master/5_nsga_results/002/convergence_data.csv
✅ Convergence plot saved to /Users/petertunali/Documents/GitHub/Battery_Optimisation/5_nsga/5_nsga_scripts_master/5_nsga_results/002/convergence_plot.png


<Figure size 1200x800 with 0 Axes>

In [14]:
# Cell 12: Analyze the best solutions in detail
print("\nAnalyzing best solutions in detail...")

# Analyze best IRR solution
best_irr_idx = df['IRR'].idxmax()
best_irr_battery = df.loc[best_irr_idx, 'battery_kwh']
best_irr_pv = df.loc[best_irr_idx, 'additional_pv_kw']

print(f"Best IRR Solution:")
print(f"Battery: {best_irr_battery:.2f} kWh")
print(f"Additional PV: {best_irr_pv:.2f} kW")
print(f"Total System PV: {existing_pv['system_capacity_kw'] + best_irr_pv:.2f} kW")
print(f"IRR: {df.loc[best_irr_idx, 'IRR']*100:.2f}%")
print(f"NPV: ${df.loc[best_irr_idx, 'NPV']:,.2f}")

# PV allocation for best IRR
print("\nPV Allocation for Best IRR:")
for option in pv_options:
    option_name = option['name']
    option_kw = df.loc[best_irr_idx, f'{option_name}_kw']
    if option_kw > 0:
        print(f"  - {option_name}: {option_kw:.2f} kW")

# Analyze best NPV solution
best_npv_idx = df['NPV'].idxmax()
best_npv_battery = df.loc[best_npv_idx, 'battery_kwh']
best_npv_pv = df.loc[best_npv_idx, 'additional_pv_kw']

print(f"\nBest NPV Solution:")
print(f"Battery: {best_npv_battery:.2f} kWh")
print(f"Additional PV: {best_npv_pv:.2f} kW")
print(f"Total System PV: {existing_pv['system_capacity_kw'] + best_npv_pv:.2f} kW")
print(f"IRR: {df.loc[best_npv_idx, 'IRR']*100:.2f}%")
print(f"NPV: ${df.loc[best_npv_idx, 'NPV']:,.2f}")

# PV allocation for best NPV
print("\nPV Allocation for Best NPV:")
for option in pv_options:
    option_name = option['name']
    option_kw = df.loc[best_npv_idx, f'{option_name}_kw']
    if option_kw > 0:
        print(f"  - {option_name}: {option_kw:.2f} kW")


Analyzing best solutions in detail...
Best IRR Solution:
Battery: 0.00 kWh
Additional PV: 11.41 kW
Total System PV: 21.41 kW
IRR: 15.52%
NPV: $15,634.57

PV Allocation for Best IRR:
  - accommodation_block: 11.41 kW

Best NPV Solution:
Battery: 18.02 kWh
Additional PV: 50.54 kW
Total System PV: 60.54 kW
IRR: 11.25%
NPV: $40,768.89

PV Allocation for Best NPV:
  - accommodation_block: 33.00 kW
  - small_shed: 10.00 kW
  - ground_mounted: 7.54 kW


In [15]:
# Cell 13: Detailed simulation of best IRR solution
print("\nRunning detailed simulation for best IRR solution...")
start_time = time.time()

# Create detailed configuration for best IRR solution
best_irr_battery = df.loc[best_irr_idx, 'battery_kwh']
best_irr_pv = df.loc[best_irr_idx, 'additional_pv_kw']
best_irr_allocated_pv = allocate_pv_capacity(best_irr_pv, pv_options)

# Add existing PV
best_irr_all_pv = [existing_pv.copy()]
for pv_config in best_irr_allocated_pv:
    if pv_config['system_capacity_kw'] > 0:
        best_irr_all_pv.append(pv_config)

# Simulate combined PV profile
best_irr_pv_profile = simulate_multi_year_pv(
    weather_files=weather_files,
    roof_params=best_irr_all_pv,
    repeats_per_file=10,
    start_years=start_years
)

# Simulate battery dispatch
best_irr_dispatch, best_irr_totals = simulate_battery_dispatch(
    pv_gen=best_irr_pv_profile['simulated_kwh'],
    demand=demand_profile,
    battery_kwh=best_irr_battery,
    battery_kw=best_irr_battery * 0.5,
    roundtrip_eff=0.9,
    min_soc_pct=0.05,
    annual_deg_rate=0.01,
    grid_emission_rate=0.81
)

elapsed = time.time() - start_time
print(f"Detailed simulation completed in {elapsed:.1f} seconds ({timedelta(seconds=int(elapsed))})")

# Calculate financial metrics 
# First, calculate PV capital costs
pv_capital_cost = 0
for pv_config in best_irr_allocated_pv:
    capacity = pv_config['system_capacity_kw']
    cost_per_kw = calculate_pv_cost(capacity, pv_config['cost_multiplier'])
    pv_capital_cost += capacity * cost_per_kw

# Add fixed installation cost of $1000 if any additional PV
if best_irr_pv > 0:
    pv_capital_cost += 1000

# Calculate battery cost
if best_irr_battery > 0:
    battery_cost_per_kwh = 977.54 * np.exp(-0.004 * best_irr_battery)
    battery_cost_per_kwh = max(600, battery_cost_per_kwh)
    battery_capital_cost = best_irr_battery * (battery_cost_per_kwh + 174.0)  # Including installation
else:
    battery_capital_cost = 0

# Calculate annual maintenance costs
pv_maintenance_cost = 250  # $250 per year for PV (instead of per kW per day)
battery_maintenance_cost = 0  # No maintenance cost for battery as requested

# Calculate annual electricity costs with 3% escalation
annual_costs = []
total_cost = 0
total_import_cost = 0
total_export_revenue = 0

for year in range(30):
    year_escalation = (1 + escalation_rate)**year
    peak_rate = base_peak_rate * year_escalation
    offpeak_rate = base_offpeak_rate * year_escalation
    export_rate = base_export_rate * year_escalation
    
    # Get annual values (assume equal distribution over 30 years)
    annual_peak_import = best_irr_totals['total_grid_import_peak'] / 30
    annual_offpeak_import = best_irr_totals['total_grid_import_offpeak'] / 30
    annual_export = best_irr_totals['total_pv_export'] / 30
    
    # Calculate this year's costs
    import_cost = annual_peak_import * peak_rate + annual_offpeak_import * offpeak_rate
    export_revenue = annual_export * export_rate
    net_cost = import_cost - export_revenue + pv_maintenance_cost
    
    annual_costs.append({
        'year': year + 1,
        'peak_rate': peak_rate,
        'offpeak_rate': offpeak_rate,
        'export_rate': export_rate,
        'import_cost': import_cost,
        'export_revenue': export_revenue,
        'maintenance_cost': pv_maintenance_cost,
        'net_cost': net_cost
    })
    
    total_cost += net_cost
    total_import_cost += import_cost
    total_export_revenue += export_revenue

# Save detailed annual costs
annual_costs_df = pd.DataFrame(annual_costs)
annual_costs_df.to_csv(run_dir / 'best_irr_annual_costs.csv', index=False)
print(f"✅ Annual cost breakdown saved to {run_dir/'best_irr_annual_costs.csv'}")

# Save battery metrics
battery_metrics = {
    'battery_size_kwh': best_irr_battery,
    'battery_power_kw': best_irr_battery * 0.5,
    'battery_cycles': best_irr_totals['battery_cycles'],
    'final_degradation_pct': best_irr_totals['final_degradation_pct']
}
pd.DataFrame([battery_metrics]).to_csv(run_dir / 'best_irr_battery_metrics.csv', index=False)
print(f"✅ Battery metrics saved to {run_dir/'best_irr_battery_metrics.csv'}")

# Save PV allocation details
pv_allocation = []
for pv_config in best_irr_all_pv:
    pv_allocation.append({
        'name': pv_config['name'],
        'capacity_kw': pv_config['system_capacity_kw'],
        'tilt': pv_config['tilt'],
        'azimuth': pv_config['azimuth'],
        'shading': pv_config['shading'],
        'array_type': pv_config['array_type'],
        'cost_multiplier': pv_config.get('cost_multiplier', 1.0)
    })
pv_allocation_df = pd.DataFrame(pv_allocation)
pv_allocation_df.to_csv(run_dir / 'best_irr_pv_allocation.csv', index=False)
print(f"✅ PV allocation details saved to {run_dir/'best_irr_pv_allocation.csv'}")

# Save full 30-year time series 
best_irr_dispatch.to_csv(run_dir / 'best_irr_30yr_timeseries.csv')
print(f"✅ 30-year time series saved to {run_dir/'best_irr_30yr_timeseries.csv'}")

# Save summary of results
summary = {
    'total_pv_capacity_kw': existing_pv['system_capacity_kw'] + best_irr_pv,
    'additional_pv_capacity_kw': best_irr_pv,
    'battery_capacity_kwh': best_irr_battery,
    'pv_capital_cost': pv_capital_cost,
    'battery_capital_cost': battery_capital_cost,
    'total_capital_cost': pv_capital_cost + battery_capital_cost,
    'annual_pv_maintenance': pv_maintenance_cost,
    'irr': df.loc[best_irr_idx, 'IRR'],
    'npv': df.loc[best_irr_idx, 'NPV'],
    'total_generation_kwh': best_irr_pv_profile['simulated_kwh'].sum(),
    'total_demand_kwh': best_irr_totals['total_demand'],
    'total_pv_used_kwh': best_irr_totals['total_pv_used'],
    'total_battery_discharge_kwh': best_irr_totals['total_battery_discharge'],
    'total_grid_import_kwh': best_irr_totals['total_grid_import_peak'] + best_irr_totals['total_grid_import_offpeak'],
    'total_pv_export_kwh': best_irr_totals['total_pv_export'],
    'renewable_fraction': best_irr_totals['renewable_fraction'],
    'self_consumption_rate': best_irr_totals['self_consumption_rate'],
    'total_grid_emissions_kg': best_irr_totals['total_grid_emissions'],
    'total_import_cost': total_import_cost,
    'total_export_revenue': total_export_revenue,
    'total_electricity_cost': total_cost
}
pd.DataFrame([summary]).to_csv(run_dir / 'best_irr_summary.csv', index=False)
print(f"✅ Summary results saved to {run_dir/'best_irr_summary.csv'}")

# Create a text file with the key results in a readable format
with open(run_dir / 'best_irr_results.txt', 'w') as f:
    f.write("===============================================\n")
    f.write("BATTERY & PV OPTIMIZATION RESULTS (BEST IRR)\n")
    f.write("===============================================\n\n")
    
    f.write("SYSTEM CONFIGURATION\n")
    f.write("-------------------\n")
    f.write(f"Existing PV Capacity: {existing_pv['system_capacity_kw']:.2f} kW\n")
    f.write(f"Additional PV Capacity: {best_irr_pv:.2f} kW\n")
    f.write(f"Total System PV Capacity: {existing_pv['system_capacity_kw'] + best_irr_pv:.2f} kW\n")
    f.write(f"Battery Size: {best_irr_battery:.2f} kWh\n")
    f.write(f"Battery Power: {best_irr_battery * 0.5:.2f} kW (0.5C rate)\n\n")
    
    f.write("PV ALLOCATION\n")
    f.write("------------\n")
    for pv_config in best_irr_all_pv:
        f.write(f"{pv_config['name']}: {pv_config['system_capacity_kw']:.2f} kW\n")
        f.write(f"  Tilt: {pv_config['tilt']}°, Azimuth: {pv_config['azimuth']}°, Shading: {pv_config['shading']}%\n")
    f.write("\n")
    
    f.write("FINANCIAL RESULTS\n")
    f.write("-----------------\n")
    f.write(f"PV Capital Cost: ${pv_capital_cost:,.2f}\n")
    f.write(f"Battery Capital Cost: ${battery_capital_cost:,.2f}\n")
    f.write(f"Total Capital Cost: ${pv_capital_cost + battery_capital_cost:,.2f}\n")
    f.write(f"Annual PV Maintenance: ${pv_maintenance_cost:,.2f}\n")
    f.write(f"IRR: {df.loc[best_irr_idx, 'IRR']*100:.2f}%\n")
    f.write(f"NPV: ${df.loc[best_irr_idx, 'NPV']:,.2f}\n\n")
    
    f.write("ENERGY RESULTS (30 YEARS)\n")
    f.write("-----------------------\n")
    f.write(f"Total Generation: {best_irr_pv_profile['simulated_kwh'].sum():,.2f} kWh\n")
    f.write(f"Total Demand: {best_irr_totals['total_demand']:,.2f} kWh\n")
    f.write(f"PV Self-Consumed: {best_irr_totals['total_pv_used']:,.2f} kWh\n")
    f.write(f"Battery Discharge: {best_irr_totals['total_battery_discharge']:,.2f} kWh\n")
    f.write(f"Grid Import: {best_irr_totals['total_grid_import_peak'] + best_irr_totals['total_grid_import_offpeak']:,.2f} kWh\n")
    f.write(f"PV Export: {best_irr_totals['total_pv_export']:,.2f} kWh\n")
    f.write(f"Renewable Fraction: {best_irr_totals['renewable_fraction']*100:.2f}%\n")
    f.write(f"Self-Consumption Rate: {best_irr_totals['self_consumption_rate']*100:.2f}%\n")
    f.write(f"Grid Emissions: {best_irr_totals['total_grid_emissions']:,.2f} kg CO2e\n\n")
    
    f.write("BATTERY PERFORMANCE\n")
    f.write("------------------\n")
    f.write(f"Cycles Over 30 Years: {best_irr_totals['battery_cycles']:,.2f}\n")
    f.write(f"Final Capacity Degradation: {best_irr_totals['final_degradation_pct']:.2f}%\n\n")
    
    f.write("ELECTRICITY COSTS (30 YEARS)\n")
    f.write("--------------------------\n")
    f.write(f"Total Import Cost: ${total_import_cost:,.2f}\n")
    f.write(f"Total Export Revenue: ${total_export_revenue:,.2f}\n")
    f.write(f"Net Electricity Cost: ${total_import_cost - total_export_revenue:,.2f}\n")
    f.write(f"Total Cost (incl. maintenance): ${total_cost:,.2f}\n\n")
    
    f.write("NOTES\n")
    f.write("-----\n")
    f.write("- Electricity prices escalate at 3% per year\n")
    f.write(f"- Initial electricity rates: Peak = ${base_peak_rate:.5f}/kWh, Off-peak = ${base_offpeak_rate:.5f}/kWh, Export = ${base_export_rate:.5f}/kWh\n")
    f.write("- PV cost calculated using formula: y = 1047.3 * e^(-0.002*x) with minimum $750/kW\n")
    f.write("- Ground-mounted PV has 20% cost premium\n")
    f.write("- Battery cost calculated using formula: y = 977.54 * e^(-0.004*x) with minimum $600/kWh\n")
    f.write("- Battery round-trip efficiency: 90%\n")
    f.write("- Battery minimum state of charge: 5%\n")

print(f"✅ Detailed results saved to {run_dir/'best_irr_results.txt'}")



Running detailed simulation for best IRR solution...
Detailed simulation completed in 1.4 seconds (0:00:01)
✅ Annual cost breakdown saved to /Users/petertunali/Documents/GitHub/Battery_Optimisation/5_nsga/5_nsga_scripts_master/5_nsga_results/002/best_irr_annual_costs.csv
✅ Battery metrics saved to /Users/petertunali/Documents/GitHub/Battery_Optimisation/5_nsga/5_nsga_scripts_master/5_nsga_results/002/best_irr_battery_metrics.csv
✅ PV allocation details saved to /Users/petertunali/Documents/GitHub/Battery_Optimisation/5_nsga/5_nsga_scripts_master/5_nsga_results/002/best_irr_pv_allocation.csv
✅ 30-year time series saved to /Users/petertunali/Documents/GitHub/Battery_Optimisation/5_nsga/5_nsga_scripts_master/5_nsga_results/002/best_irr_30yr_timeseries.csv
✅ Summary results saved to /Users/petertunali/Documents/GitHub/Battery_Optimisation/5_nsga/5_nsga_scripts_master/5_nsga_results/002/best_irr_summary.csv
✅ Detailed results saved to /Users/petertunali/Documents/GitHub/Battery_Optimisatio

In [16]:
# Cell 14: Create daily profile plots
print("\nCreating daily profile plots...")

print("\nCreating daily profile plots...")

# Summer day
summer_date = pd.Timestamp(f"{demand_profile.index[0].year}-01-15")
summer_mask = (best_irr_dispatch.index.date == summer_date.date())
summer_data = best_irr_dispatch[summer_mask]

plt.figure(figsize=(12, 6))
plt.plot(summer_data.index.strftime('%H:%M'), summer_data['pv_gen'], 'orange', label='PV Generation')
plt.plot(summer_data.index.strftime('%H:%M'), summer_data['demand'], 'blue', label='Demand')
plt.plot(summer_data.index.strftime('%H:%M'), summer_data['battery_discharge'], 'green', label='Battery Discharge')
plt.plot(summer_data.index.strftime('%H:%M'), summer_data['grid_import_peak'] + summer_data['grid_import_offpeak'], 'red', label='Grid Import')
plt.fill_between(summer_data.index.strftime('%H:%M'), summer_data['battery_soc'], alpha=0.3, color='green', label='Battery SOC')
plt.xticks(rotation=90)
plt.title(f'Summer Day Profile: {summer_date.strftime("%B %d")}')
plt.xlabel('Time of Day')
plt.ylabel('Energy (kWh)')
plt.legend()
plt.tight_layout()
plt.savefig(run_dir / "summer_day_profile.png", dpi=300)
plt.close()

# Winter day
winter_date = pd.Timestamp(f"{demand_profile.index[0].year}-07-15")
winter_mask = (best_irr_dispatch.index.date == winter_date.date())
winter_data = best_irr_dispatch[winter_mask]

plt.figure(figsize=(12, 6))
plt.plot(winter_data.index.strftime('%H:%M'), winter_data['pv_gen'], 'orange', label='PV Generation')
plt.plot(winter_data.index.strftime('%H:%M'), winter_data['demand'], 'blue', label='Demand')
plt.plot(winter_data.index.strftime('%H:%M'), winter_data['battery_discharge'], 'green', label='Battery Discharge')
plt.plot(winter_data.index.strftime('%H:%M'), winter_data['grid_import_peak'] + winter_data['grid_import_offpeak'], 'red', label='Grid Import')
plt.fill_between(winter_data.index.strftime('%H:%M'), winter_data['battery_soc'], alpha=0.3, color='green', label='Battery SOC')
plt.xticks(rotation=90)
plt.title(f'Winter Day Profile: {winter_date.strftime("%B %d")}')
plt.xlabel('Time of Day')
plt.ylabel('Energy (kWh)')
plt.legend()
plt.tight_layout()
plt.savefig(run_dir / "winter_day_profile.png", dpi=300)
plt.close()

print(f"✅ Daily profile plots saved to {run_dir}")


Creating daily profile plots...

Creating daily profile plots...
✅ Daily profile plots saved to /Users/petertunali/Documents/GitHub/Battery_Optimisation/5_nsga/5_nsga_scripts_master/5_nsga_results/002


In [18]:
# Cell 16: Comprehensive Financial Analysis and Verification
print("\nPerforming comprehensive financial analysis and verification...")

# Define discount rate here at the beginning
discount_rate = 0.07  # 7% discount rate

# Create a new dataframe for the comprehensive financial summary
fin_summary = {
    'metric': [],
    'value': [],
    'unit': [],
    'description': []
}

# Add basic system information
fin_summary['metric'].extend(['total_pv_capacity', 'additional_pv_capacity', 'battery_capacity'])
fin_summary['value'].extend([
    existing_pv['system_capacity_kw'] + best_irr_pv,
    best_irr_pv,
    best_irr_battery
])
fin_summary['unit'].extend(['kW', 'kW', 'kWh'])
fin_summary['description'].extend([
    'Total PV system capacity (existing + additional)',
    'Additional PV capacity beyond existing system',
    'Battery energy storage capacity'
])

# Add energy metrics
fin_summary['metric'].extend([
    'total_demand', 'total_pv_used', 'total_battery_discharge',
    'total_grid_import_peak', 'total_grid_import_offpeak', 'total_pv_export',
    'total_grid_emissions', 'renewable_fraction', 'grid_fraction',
    'self_consumption_rate'
])
fin_summary['value'].extend([
    best_irr_totals['total_demand'],
    best_irr_totals['total_pv_used'],
    best_irr_totals['total_battery_discharge'],
    best_irr_totals['total_grid_import_peak'],
    best_irr_totals['total_grid_import_offpeak'],
    best_irr_totals['total_pv_export'],
    best_irr_totals['total_grid_emissions'],
    best_irr_totals['renewable_fraction'],
    1 - best_irr_totals['renewable_fraction'],
    best_irr_totals['self_consumption_rate']
])
fin_summary['unit'].extend([
    'kWh', 'kWh', 'kWh', 'kWh', 'kWh', 'kWh', 'kg CO2e', 'fraction', 'fraction', 'fraction'
])
fin_summary['description'].extend([
    'Total electricity demand over 30 years',
    'PV generation used directly to satisfy demand',
    'Energy discharged from battery storage',
    'Grid electricity imported during peak periods',
    'Grid electricity imported during off-peak periods',
    'Excess PV generation exported to grid',
    'CO2 emissions from grid electricity',
    'Fraction of demand met by renewable sources',
    'Fraction of demand met by grid imports',
    'Fraction of PV generation consumed on-site'
])

# Add financial metrics
fin_summary['metric'].extend([
    'pv_capex', 'battery_capex', 'total_capex',
    'annual_maintenance', 'total_maintenance',
    'npv', 'irr', 'simple_payback'
])

# Calculate battery cost
if best_irr_battery > 0:
    battery_cost_per_kwh = 977.54 * np.exp(-0.004 * best_irr_battery)
    battery_cost_per_kwh = max(600.0, battery_cost_per_kwh)
    battery_capex = best_irr_battery * (battery_cost_per_kwh + 174.0)
else:
    battery_capex = 0

# Calculate PV capital costs
pv_capital_cost = 0
if best_irr_pv > 0:
    # Calculate costs for each allocated PV component
    for pv_config in best_irr_allocated_pv:
        capacity = pv_config['system_capacity_kw']
        cost_multiplier = pv_config.get('cost_multiplier', 1.0)
        cost_per_kw = calculate_pv_cost(capacity, cost_multiplier)
        pv_capital_cost += capacity * cost_per_kw
    
    # Add fixed installation cost
    pv_capital_cost += 1000.0

annual_maintenance = 250  # $250 per year for PV maintenance
total_maintenance = annual_maintenance * 30

fin_summary['value'].extend([
    pv_capital_cost,
    battery_capex,
    pv_capital_cost + battery_capex,
    annual_maintenance,
    total_maintenance,
    df.loc[best_irr_idx, 'NPV'],
    df.loc[best_irr_idx, 'IRR'] if pd.notna(df.loc[best_irr_idx, 'IRR']) else None,
    # Simple payback calculation - compare to PV-only
    (pv_capital_cost + battery_capex) / (annual_savings_vs_pv_only if 'annual_savings_vs_pv_only' in locals() and annual_savings_vs_pv_only > 0 else float('inf'))
])

fin_summary['unit'].extend([
    '$', '$', '$', '$/year', '$', '$', 'fraction', 'years'
])

fin_summary['description'].extend([
    'Capital cost for additional PV capacity',
    'Capital cost for battery storage',
    'Total capital expenditure',
    'Annual maintenance cost',
    'Total maintenance cost over 30 years',
    'Net Present Value of investment',
    'Internal Rate of Return',
    'Simple payback period against PV-only baseline'
])

# Create and save the dataframe
fin_summary_df = pd.DataFrame(fin_summary)
fin_summary_df.to_csv(run_dir / 'financial_summary.csv', index=False)
print(f"✅ Comprehensive financial summary saved to {run_dir/'financial_summary.csv'}")

# Calculate cash flows for incremental analysis
print("\nVerifying incremental financial analysis:")
print("This analysis compares the proposed system (existing PV + new PV + battery)")
print("against the current baseline (existing PV only - $8,246.44/year)")

# Set up escalation for electricity prices
annual_costs = []
annual_pv_only_costs = []

for year in range(30):
    # Apply escalation
    year_escalation = (1 + escalation_rate)**year
    year_peak_rate = base_peak_rate * year_escalation
    year_offpeak_rate = base_offpeak_rate * year_escalation
    year_export_rate = base_export_rate * year_escalation
    
    # PV-only baseline (with escalation)
    year_pv_only_cost = ANNUAL_PV_ONLY_COST * year_escalation
    annual_pv_only_costs.append(year_pv_only_cost)
    
    # Proposed system
    # Get annual values (assume equal distribution)
    annual_peak_import = best_irr_totals['total_grid_import_peak'] / 30
    annual_offpeak_import = best_irr_totals['total_grid_import_offpeak'] / 30
    annual_export = best_irr_totals['total_pv_export'] / 30
    
    # Calculate this year's costs
    year_import_cost = annual_peak_import * year_peak_rate + annual_offpeak_import * year_offpeak_rate
    year_export_revenue = annual_export * year_export_rate
    year_net_cost = year_import_cost - year_export_revenue + annual_maintenance
    
    annual_costs.append({
        'year': year + 1,
        'escalation_factor': year_escalation,
        'peak_rate': year_peak_rate,
        'offpeak_rate': year_offpeak_rate,
        'export_rate': year_export_rate,
        'import_cost': year_import_cost,
        'export_revenue': year_export_revenue,
        'maintenance': annual_maintenance,
        'net_cost': year_net_cost,
        'pv_only_cost': year_pv_only_cost,
        'savings_vs_pv_only': year_pv_only_cost - year_net_cost
    })

# Calculate NPV
cash_flows = [-(pv_capital_cost + battery_capex)]  # Initial investment
for year_data in annual_costs:
    cash_flows.append(year_data['savings_vs_pv_only'])

# Calculate NPV and IRR
npv = sum(cf / (1 + discount_rate)**(i) for i, cf in enumerate(cash_flows))
try:
    # Calculate IRR
    from scipy import optimize
    def npv_func(rate):
        return sum(cf / (1 + rate)**(i) for i, cf in enumerate(cash_flows))
    
    try:
        irr_value = optimize.newton(npv_func, 0.05)
    except:
        try:
            irr_value = optimize.brentq(npv_func, -0.9999, 2.0)
        except:
            irr_value = None
except:
    irr_value = None

# Print verification details
print(f"\nCalculated NPV: ${npv:,.2f}")
print(f"Calculated IRR: {irr_value*100:.2f}%" if irr_value is not None else "Calculated IRR: N/A")
print(f"Calculated Simple Payback: {(pv_capital_cost + battery_capex) / annual_costs[0]['savings_vs_pv_only']:.2f} years" 
      if annual_costs[0]['savings_vs_pv_only'] > 0 else "Calculated Simple Payback: N/A (no positive savings)")

# Show first few years of cash flow
print("\nIncremental Cash Flow Analysis (first 5 years):")
print("Year | Proposed System | PV-Only Baseline | Savings | Discounted Savings | Cumulative NPV")
print("----|----------------|-----------------|---------|-------------------|---------------")

cumulative_npv = cash_flows[0]  # Start with initial investment
print(f"   0 | ${0:13,.2f} | ${0:15,.2f} | ${cash_flows[0]:7,.2f} | ${cash_flows[0]:17,.2f} | ${cumulative_npv:13,.2f}")

for i, year_data in enumerate(annual_costs[:5]):  # Show first 5 years
    year = i + 1
    disc_savings = year_data['savings_vs_pv_only'] / (1 + discount_rate)**year
    cumulative_npv += disc_savings
    
    print(f"{year:4d} | ${year_data['net_cost']:13,.2f} | ${year_data['pv_only_cost']:15,.2f} | " +
          f"${year_data['savings_vs_pv_only']:7,.2f} | ${disc_savings:17,.2f} | ${cumulative_npv:13,.2f}")

# Show electricity price escalation
print("\nElectricity Price Escalation (first 5 years with 3% annual increase):")
print("Year | Peak Rate | Off-Peak Rate | Export Rate")
print("----|-----------|---------------|------------")
for i, year_data in enumerate(annual_costs[:5]):  # Show first 5 years
    print(f"{year_data['year']:4d} | ${year_data['peak_rate']:8,.5f} | ${year_data['offpeak_rate']:12,.5f} | ${year_data['export_rate']:10,.5f}")

# Save the detailed cash flow
annual_costs_df = pd.DataFrame(annual_costs)
annual_costs_df.to_csv(run_dir / 'annual_cash_flows.csv', index=False)
print(f"\n✅ Detailed annual cash flows saved to {run_dir/'annual_cash_flows.csv'}")

# Create an improved text file summary
with open(run_dir / 'financial_analysis.txt', 'w') as f:
    f.write("===============================================\n")
    f.write("BATTERY & PV OPTIMIZATION FINANCIAL ANALYSIS\n")
    f.write("===============================================\n\n")
    
    f.write("SYSTEM CONFIGURATION\n")
    f.write("-------------------\n")
    f.write(f"Existing PV Capacity: {existing_pv['system_capacity_kw']:.2f} kW\n")
    f.write(f"Additional PV Capacity: {best_irr_pv:.2f} kW\n")
    f.write(f"Total System PV Capacity: {existing_pv['system_capacity_kw'] + best_irr_pv:.2f} kW\n")
    f.write(f"Battery Size: {best_irr_battery:.2f} kWh\n")
    f.write(f"Battery Power: {best_irr_battery * 0.5:.2f} kW (0.5C rate)\n\n")
    
    f.write("PV ALLOCATION\n")
    f.write("------------\n")
    f.write(f"Existing System: {existing_pv['system_capacity_kw']:.2f} kW\n")
    for pv_config in best_irr_allocated_pv:
        f.write(f"{pv_config['name']}: {pv_config['system_capacity_kw']:.2f} kW\n")
        f.write(f"  Tilt: {pv_config['tilt']}°, Azimuth: {pv_config['azimuth']}°, Shading: {pv_config['shading']}%\n")
    f.write("\n")
    
    f.write("FINANCIAL RESULTS\n")
    f.write("-----------------\n")
    f.write(f"PV Capital Cost: ${pv_capital_cost:,.2f}\n")
    f.write(f"Battery Capital Cost: ${battery_capex:,.2f}\n")
    f.write(f"Total Capital Cost: ${pv_capital_cost + battery_capex:,.2f}\n")
    f.write(f"Annual PV Maintenance: ${annual_maintenance:,.2f}\n")
    f.write(f"IRR: {irr_value*100:.2f}%\n" if irr_value is not None else "IRR: N/A\n")
    f.write(f"NPV: ${npv:,.2f}\n")
    if annual_costs[0]['savings_vs_pv_only'] > 0:
        f.write(f"Simple Payback: {(pv_capital_cost + battery_capex) / annual_costs[0]['savings_vs_pv_only']:.2f} years\n\n")
    else:
        f.write(f"Simple Payback: N/A (no positive savings)\n\n")
    
    f.write("ENERGY RESULTS (30 YEARS)\n")
    f.write("-----------------------\n")
    f.write(f"Total Generation: {best_irr_pv_profile['simulated_kwh'].sum():,.2f} kWh\n")
    f.write(f"Total Demand: {best_irr_totals['total_demand']:,.2f} kWh\n")
    f.write(f"PV Self-Consumed: {best_irr_totals['total_pv_used']:,.2f} kWh\n")
    f.write(f"Battery Discharge: {best_irr_totals['total_battery_discharge']:,.2f} kWh\n")
    f.write(f"Grid Import: {best_irr_totals['total_grid_import_peak'] + best_irr_totals['total_grid_import_offpeak']:,.2f} kWh\n")
    f.write(f"PV Export: {best_irr_totals['total_pv_export']:,.2f} kWh\n")
    f.write(f"Renewable Fraction: {best_irr_totals['renewable_fraction']*100:.2f}%\n")
    f.write(f"Self-Consumption Rate: {best_irr_totals['self_consumption_rate']*100:.2f}%\n")
    f.write(f"Grid Emissions: {best_irr_totals['total_grid_emissions']:,.2f} kg CO2e\n\n")
    
    f.write("BATTERY PERFORMANCE\n")
    f.write("------------------\n")
    f.write(f"Cycles Over 30 Years: {best_irr_totals['battery_cycles']:,.2f}\n")
    f.write(f"Final Capacity Degradation: {best_irr_totals['final_degradation_pct']:.2f}%\n\n")
    
    f.write("INCREMENTAL ANALYSIS VS. PV-ONLY\n")
    f.write("------------------------------\n")
    f.write(f"Current Annual Cost (PV-only): ${ANNUAL_PV_ONLY_COST:,.2f}\n")
    f.write(f"First Year Annual Cost (Proposed): ${annual_costs[0]['net_cost']:,.2f}\n")
    f.write(f"First Year Savings: ${annual_costs[0]['savings_vs_pv_only']:,.2f}\n")
    f.write(f"Total 30-Year Savings (undiscounted): ${sum(year_data['savings_vs_pv_only'] for year_data in annual_costs):,.2f}\n\n")
    
    f.write("NOTES\n")
    f.write("-----\n")
    f.write("- Incremental analysis compares against existing PV-only baseline\n")
    f.write("- Electricity prices escalate at 3% per year\n")
    f.write(f"- Initial electricity rates: Peak = ${base_peak_rate:.5f}/kWh, Off-peak = ${base_offpeak_rate:.5f}/kWh, Export = ${base_export_rate:.5f}/kWh\n")
    f.write("- PV cost calculated using formula: y = 1047.3 * e^(-0.002*x) with minimum $750/kW\n")
    f.write("- Ground-mounted PV has 20% cost premium\n")
    f.write("- Battery cost calculated using formula: y = 977.54 * e^(-0.004*x) with minimum $600/kWh\n")
    f.write("- No battery maintenance costs as per requirements\n")
    f.write("- PV maintenance cost: $250 per year\n")
    f.write("- Discount rate for NPV calculation: 7%\n")

print(f"✅ Detailed financial analysis saved to {run_dir/'financial_analysis.txt'}")
print("\nFinancial analysis complete!")


Performing comprehensive financial analysis and verification...
✅ Comprehensive financial summary saved to /Users/petertunali/Documents/GitHub/Battery_Optimisation/5_nsga/5_nsga_scripts_master/5_nsga_results/002/financial_summary.csv

Verifying incremental financial analysis:
This analysis compares the proposed system (existing PV + new PV + battery)
against the current baseline (existing PV only - $8,246.44/year)

Calculated NPV: $15,612.23
Calculated IRR: 15.51%
Calculated Simple Payback: 7.96 years

Incremental Cash Flow Analysis (first 5 years):
Year | Proposed System | PV-Only Baseline | Savings | Discounted Savings | Cumulative NPV
----|----------------|-----------------|---------|-------------------|---------------
   0 | $         0.00 | $           0.00 | $-12,675.31 | $       -12,675.31 | $   -12,675.31
   1 | $     6,653.06 | $       8,246.44 | $1,593.38 | $         1,489.14 | $   -11,186.18
   2 | $     6,845.16 | $       8,493.83 | $1,648.68 | $         1,440.02 | $    -9